## 准备数据


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

2025-03-23 18:24:40.204104: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-23 18:24:40.226502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742725480.251555  827660 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742725480.258814  827660 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 18:24:40.285284: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型


In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        initializer = tf.keras.initializers.GlorotNormal()
        self.W1 = tf.Variable(initializer([28*28, 100]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([100]), dtype=tf.float32)
        self.W2 = tf.Variable(initializer([100, 10]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), dtype=tf.float32)

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])
        
        h1 = tf.matmul(x, self.W1) + self.b1
        h1_relu = tf.nn.relu(h1)
        
        logits = tf.matmul(h1_relu, self.W2) + self.b2
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

I0000 00:00:1742725484.146067  827660 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 74042 MB memory:  -> device: 0, name: NVIDIA A800 80GB PCIe, pci bus id: 0000:4f:00.0, compute capability: 8.0
I0000 00:00:1742725484.147779  827660 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 79069 MB memory:  -> device: 1, name: NVIDIA A800 80GB PCIe, pci bus id: 0000:52:00.0, compute capability: 8.0
I0000 00:00:1742725484.149345  827660 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 79069 MB memory:  -> device: 2, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:56:00.0, compute capability: 8.0
I0000 00:00:1742725484.150698  827660 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 79069 MB memory:  -> device: 3, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:57:00.0, compute capability: 8.0
I0000 00:00:1742725484.152285  827660 gpu_device.cc:2022] Create

## 计算 loss


In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练


In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.2934412 ; accuracy 0.72705
epoch 1 : loss 1.2891632 ; accuracy 0.7281167
epoch 2 : loss 1.2849207 ; accuracy 0.72915
epoch 3 : loss 1.2806945 ; accuracy 0.72996664
epoch 4 : loss 1.2764939 ; accuracy 0.7309167
epoch 5 : loss 1.2723216 ; accuracy 0.73183334
epoch 6 : loss 1.2681676 ; accuracy 0.73286664
epoch 7 : loss 1.2640524 ; accuracy 0.73368335
epoch 8 : loss 1.2599543 ; accuracy 0.73475
epoch 9 : loss 1.2558646 ; accuracy 0.73585
epoch 10 : loss 1.2518164 ; accuracy 0.7367167
epoch 11 : loss 1.2477885 ; accuracy 0.7377833
epoch 12 : loss 1.2437929 ; accuracy 0.73878336
epoch 13 : loss 1.2398264 ; accuracy 0.7399
epoch 14 : loss 1.235879 ; accuracy 0.74086666
epoch 15 : loss 1.2319466 ; accuracy 0.7417167
epoch 16 : loss 1.2280463 ; accuracy 0.7428
epoch 17 : loss 1.2241628 ; accuracy 0.74371666
epoch 18 : loss 1.2203064 ; accuracy 0.7445
epoch 19 : loss 1.2164738 ; accuracy 0.74518335
epoch 20 : loss 1.212673 ; accuracy 0.74598336
epoch 21 : loss 1.2088629 ; accur